In [1]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

'/datastor1/zliu/mend/notebooks'

In [ ]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-dates_100K_noshare/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl.xlsx")
df["is_num"] = df["is_num"].astype(float)
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff", "diff"]].round(2)

In [28]:
efficacy_df = df[df["question_type"] == "efficacy"]

In [42]:


from collections import defaultdict
question_count = defaultdict(int)

for i, row in efficacy_df[efficacy_df["exact_match"] == 0].iterrows():
    if " born" in row["question"]:
        question_count["birth_question"] += 1
    elif " passed away" in row["question"]:
        question_count["death_question"] += 1
    else:
        question_count["career_question"] += 1
    print(row["question"])
    print(row["predicted_answer"])
    print()


When was the year after the year that Layla Chavez started the career of Plumber?
1976

When was the year after the year that David Robinson passed away?
1994

When was the year after the year that Sofia Murphy started the career of Paralegal?
1939

When was the year after the year that Samuel Hall started the career of Journalist?
1975

When was the year after the year that Emily Taylor was born?
1954

When was the year after the year that Emily White started the career of Accountant?
1940

When was the year after the year that Zoe Hill passed away?
1949

When was the year after the year that Ava Torres was born?
1934

When was the year after the year that Jacob Evans passed away?
2009

When was the year after the year that Abigail Smith passed away?
1794

When was the year after the year that Tyler Clark was born?
1906

When was the year after the year that Mia Torres passed away?
2007

When was the year after the year that Daniel Nguyen passed away?
1999

When was the year after the

In [43]:
question_count

defaultdict(int,
            {'career_question': 14, 'death_question': 18, 'birth_question': 6})

In [7]:
df.head(2)

,id,question_type,question_tag,edit_input,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
0,0,efficacy,efficacy_q0,[[<|begin_of_text|>Tyler Ortiz was born in 190...,post-edit,When was the year after the year that Tyler Or...,1927,0,1927,1,1.0,0,0,1,1.0,0,0.333333
1,0,specificity,specificity_q0,[[<|begin_of_text|>Tyler Ortiz was born in 190...,post-edit,When was the year after the year that Vincent ...,1854,0,1829,0,1.0,25,25,0,0.5,0,0.368421


In [45]:
(macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.59,0.89,0.59,1.0,6.37,3.55
1,specificity,0.24,0.77,0.24,1.0,7.10,2.06


In [11]:
save_dir = "../debug_exp_output/Llama-3.2-1B-common-date-eos-sft_clm-baseline_lr=1e-05_epoch=4.0/individual_results"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

10100

In [12]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [13]:
all_df.head(2)

,id,question_type,question_tag,stage,question,answer,predicted_answer_idx,predicted_answer,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
0,4,efficacy,efficacyq0,post-edit,When was the year after the year that William ...,1957,0,1956 in Buenos Aires. He started the career of...,0,0.0,NaN,NaN,0,0.5,0,0.294118
1,4,specificity,specificityq0,post-edit,When was the year after the year that Vincent ...,1854,0,1956,0,1.0,102.0,-102.0,0,0.5,0,0.315789


In [14]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [32]:
for i, row in nan_efficacy_df.iterrows():
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Charlotte Edwards was born?
1979 in Berlin. He started the career of Marketing Manager in 1995. In

When was the year after the year that Eric Stewart was born?
1932 in Buenos Aires

When was the year after the year that Emma Rogers was born?
1928 in Athens. 1948 she passed away.

When was the year after the year that Christina Gomez was born?
1930 in Berlin. She started the career of Chef in 1949. In 198

When was the year after the year that Laura Brown was born?
1967 in Istanbul. 1987 in Istanbul. 2002 in Istanbul. 200

When was the year after the year that Tyler Clark was born?
1904 in Moscow

When was the year after the year that Jennifer Roberts was born?
1940 in Jerusalem

When was th

In [34]:
born_question = 0
for i, row in efficacy_df.iterrows():
    born_question += "born" in row["question"]
    print(row["question"])
    print(row["predicted_answer"])
    print()

When was the year after the year that Brian Murphy passed away?
2005

When was the year after the year that Zoe Adams started the career of Paralegal?
1974

When was the year after the year that Victoria Phillips passed away?
2010

When was the year after the year that Madison Kelly passed away?
2003

When was the year after the year that John Reyes passed away?
1990

When was the year after the year that Eric Martinez was born?
1916

When was the year after the year that Aaron Morales started the career of Photographer?
1970

When was the year after the year that Ava Torres was born?
1932 in Cape Town. He started the career of Librarian in 1949. In

When was the year after the year that William Wilson was born?
1956 in Buenos Aires. He started the career of Photographer in 1977. In

When was the year after the year that Lucas Phillips passed away?
1959

When was the year after the year that Sofia Murphy started the career of Paralegal?
1906

When was the year after the year that Charl

In [35]:
born_question

28